<a href="https://colab.research.google.com/github/widifadi/AutomateConversion/blob/main/ExtractShpManual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:
from google.colab import drive

# Check if drive is already mounted
import os
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Google Drive is already mounted at /content/drive")

Mounted at /content/drive


In [4]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas geopandas shapely > /dev/null 2>&1

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiPoint
import os
import numpy as np # import numpy to help filter out invalid coordinate values
import warnings # import the warnings module
import shutil

# Replace with your file ID
file_id = '1m23mu_aiwSIxo22uZuR-ROVt3hgwdeDRObge6ZFoNRA'  # extract file ID from the Google Sheet URL

# Replace with your desired output folder
output_folder = '/content/drive/MyDrive/LALIN DC/Final Upload Shp v2'
os.makedirs(output_folder, exist_ok=True)

# Build the Drive API client
drive_service = build('drive', 'v3', credentials=creds)

# Export the spreadsheet file to Excel format
request = drive_service.files().export_media(fileId=file_id, mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
# using export_media to export the file to a downloadable format, which is Excel (xlsx) in this case
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

# Load the spreadsheet into pandas
df_dict = pd.read_excel(fh, sheet_name=None) # read the downloaded excel file using pandas
fh.close()  # to prevent problems during execution

# Dictionary to store problematic records per sheet
problematic_records_dict = {}

# Process the data as before, starting from the loop that iterates over sheets
for sheet_name, df in list(df_dict.items())[2:]: # iterating through the sheets using the data from 'df', not 'sheets_dict'
    print(f"Processing sheet: {sheet_name}")

    problematic_records = []

    # Handle different geometry types based on sheet names
    if sheet_name == "4 PAGAR PENGAMAN" or sheet_name == "5 MARKA":
        geometry_with_indices = []

        for index, row in df.iterrows():
            try:
                lon1, lat1 = row['TitikKoordinatAwal_Longitude'], row['TitikKoordinatAwal_Latitude']
                lon2, lat2 = row['TitikKoordinatAkhir_Longitude'], row['TitikKoordinatAkhir_Latitude']

                # Convert valid numeric values; set invalid values to NaN manually
                def safe_float(value):
                    return float(value) if value not in ["", "-", None] else np.nan

                lon1, lat1, lon2, lat2 = map(safe_float, [lon1, lat1, lon2, lat2])

                # Check validity
                start_valid = np.isfinite(lon1) and np.isfinite(lat1)
                end_valid = np.isfinite(lon2) and np.isfinite(lat2)

                if sheet_name == "4 PAGAR PENGAMAN":
                    if start_valid and end_valid:
                        geometry_with_indices.append((index, LineString([(lon1, lat1), (lon2, lat2)])))
                    elif start_valid:
                        geometry_with_indices.append((index, LineString([(lon1, lat1), (lon1, lat1)])))  # Degenerate LineString
                        problematic_records.append(row.to_dict())
                    elif end_valid:
                        geometry_with_indices.append((index, LineString([(lon2, lat2), (lon2, lat2)])))  # Degenerate LineString
                        problematic_records.append(row.to_dict())
                    else:
                        print(f"Skipping row {index} due to completely missing coordinates in sheet {sheet_name}")
                        problematic_records.append(row.to_dict())

                elif sheet_name == "5 MARKA":
                    points = []
                    if start_valid:
                        points.append(Point(lon1, lat1))
                    if end_valid:
                        points.append(Point(lon2, lat2))

                    # If either one or both coordinates are invalid, add to problematic records
                    if len(points) < 2:  # This ensures records with 0 or 1 valid points are stored
                        problematic_records.append(row.to_dict())

                    if points:  # If at least one valid point exists
                        geometry_with_indices.append((index, MultiPoint(points)))
                    else:
                        print(f"Skipping row {index} due to completely invalid coordinates in sheet {sheet_name}")
                        problematic_records.append(row.to_dict())

            except (KeyError, TypeError):
                print(f"Skipping row {index} due to missing or invalid coordinates in sheet {sheet_name}")
                problematic_records.append(row.to_dict())
                continue

        # Extract geometries and valid indices
        geometry = [geom for _, geom in geometry_with_indices]
        valid_indices = [index for index, _ in geometry_with_indices]

        # Filter the DataFrame to keep only rows with valid geometries
        df = df.loc[valid_indices].reset_index(drop=True)

    else:
        # Create a list to store valid geometries and their corresponding indices
        geometry_with_indices = []
        for index, row in df.iterrows():
            try:
                # Filter out invalid coordinates (NaN, inf, -inf)
                lon = float(row['Longitude'])
                lat = float(row['Latitude'])

                if np.isfinite(lon) and np.isfinite(lat):
                    geometry_with_indices.append((index, Point(lon, lat)))
                else:
                    print(f"Skipping row {index} due to invalid coordinates (NaN, inf, -inf) in sheet {sheet_name}")
                    problematic_records.append(row.to_dict())

            except (KeyError, TypeError, ValueError):
                print(f"Skipping row {index} due to missing or invalid coordinates in sheet {sheet_name}")
                problematic_records.append(row.to_dict())
                continue

        # Extract geometries and valid indices
        geometry = [geom for _, geom in geometry_with_indices]
        valid_indices = [index for index, _ in geometry_with_indices]

        # Filter the DataFrame to keep only rows with valid geometries
        df = df.loc[valid_indices]

    # Store problematic records for this sheet
    if problematic_records:
        problematic_records_dict[sheet_name] = pd.DataFrame(problematic_records)

    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # Set a proper CRS

    batas_wilayah = gpd.read_file('/content/drive/MyDrive/LALIN DC/Prop. Data/batas wilayah kota/Batas_Kota_Kabupaten_JABAR.shp')
    gdf = gpd.sjoin(gdf, batas_wilayah[['geometry', 'NAMOBJ']], how="left", predicate="intersects")

    qml_folder = "/content/drive/MyDrive/LALIN DC/QML file_copy"  # Path where .qml files are stored

    for name, group in gdf.groupby(['NAMOBJ', 'Nama Ruas Jalan', 'Jenis File']):
        kabupaten_kota, jalan_name, jenis_file = name

        # Ensure directory structure
        sanitized_kabupaten = "".join(c for c in kabupaten_kota if c.isalnum() or c in (' ', '_')).strip()
        sanitized_jalan = "".join(c for c in jalan_name if c.isalnum() or c in (' ', '_')).strip()
        sanitized_jenis = "".join(c for c in jenis_file if c.isalnum() or c in (' ', '_')).strip()

        jalan_folder = os.path.join(output_folder, sanitized_kabupaten, "Jalan Prioritas", sanitized_jenis)
        os.makedirs(jalan_folder, exist_ok=True)

        # Save shapefile
        shapefile_path = os.path.join(jalan_folder, f"{sanitized_jalan}_{sheet_name}.shp")
        with warnings.catch_warnings():
          warnings.filterwarnings('ignore', message='Normalized/laundered field name')
          warnings.filterwarnings('ignore', message='Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.')
          group.to_file(shapefile_path)

        # Apply QML Style
        qml_source_file = os.path.join(qml_folder, f"{sheet_name}.qml")  # Assuming QML file follows sheet name
        qml_target_file = shapefile_path.replace(".shp", ".qml")

        if os.path.exists(qml_source_file):
            shutil.copy(qml_source_file, qml_target_file)
            print(f"Applied QML style: {qml_target_file}")
        else:
            print(f"No QML file found for {sheet_name}")


# Save all problematic records into one Excel file with multiple sheets
if problematic_records_dict:
    with pd.ExcelWriter("/content/drive/MyDrive/LALIN DC/problematic_records.xlsx") as writer:
        for sheet, df in problematic_records_dict.items():
            df.to_excel(writer, sheet_name=sheet, index=False)

    print("Saved all problematic records to 'problematic_records.xlsx' with proper sheet names.")

Download 100%.
Processing sheet: 1 RAMBU
Skipping row 1957 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 2499 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 2751 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 2752 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 2961 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3509 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3510 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3514 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3515 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3548 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3549 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3552 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping row 3553 due to missing or invalid coordinates in sheet 1 RAMBU
Skipping r

In [8]:
from google.colab import drive
import os
import geopandas as gpd
import pandas as pd
import shutil

# Mount Google Drive (if using Colab)
drive.mount('/content/drive')

# Define input shapefile path
shapefile_path = '/content/drive/MyDrive/LALIN DC/Prop. Data/shp ruas jalan provinsi jabar/SHP PROVINSI FINAL 14032023/SHP JALAN PROVINSI FINAL 14032023 FINAL.shp'

# Define output directory
output_folder = '/content/drive/MyDrive/LALIN DC/Ruas Jalan Splitted'
os.makedirs(output_folder, exist_ok=True)

# Load the shapefile
gdf = gpd.read_file(shapefile_path)

# Check if required columns exist
if 'nama jalan' not in gdf.columns or 'kabupaten/' not in gdf.columns:
    raise ValueError("The shapefile is missing required columns: 'nama jalan' or 'kabupaten/'")

# List to store data for Excel
excel_data = []

# Group by 'kabupaten/' and export each row separately
for kabupaten, group in gdf.groupby('kabupaten/'):
    kabupaten_folder = os.path.join(output_folder, kabupaten)
    os.makedirs(kabupaten_folder, exist_ok=True)

    for index, row in group.iterrows():
        jalan_name = row['nama jalan']

        # Sanitize file name (replace non-alphanumeric characters with underscores)
        sanitized_jalan = "".join(c if c.isalnum() or c in (' ', '_') else "_" for c in jalan_name).strip()

        # Define shapefile path
        shapefile_output_path = os.path.join(kabupaten_folder, f"{sanitized_jalan}.shp")

        # Convert row to a GeoDataFrame correctly
        gdf_row = gpd.GeoDataFrame.from_dict({col: [row[col]] for col in gdf.columns})
        gdf_row.set_geometry([row.geometry], inplace=True)  # Ensure geometry is correctly set
        gdf_row.set_crs(gdf.crs, inplace=True)  # Maintain CRS

        # Save as a separate shapefile
        gdf_row.to_file(shapefile_output_path)

        # Store data for Excel
        excel_data.append([jalan_name, kabupaten])

# Convert data to a sorted Pandas DataFrame
df_excel = pd.DataFrame(excel_data, columns=['Nama Jalan', 'Kabupaten']).sort_values('Nama Jalan')

# Save to Excel
excel_output_path = os.path.join(output_folder, 'jalan_kabupaten.xlsx')
df_excel.to_excel(excel_output_path, index=False)

print("Export completed successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Export completed successfully!
